<a href="https://colab.research.google.com/github/abkerper/project_gss/blob/main/CombiningVariables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [146]:
df1_og= pd.read_parquet('/content/gss_chunk_1.parquet', engine='pyarrow')
df2_og = pd.read_parquet('/content/gss_chunk_2.parquet', engine='pyarrow')
df3_og = pd.read_parquet('/content/gss_chunk_3.parquet', engine='pyarrow')

In [147]:
#make dummy variables of the data
df1 = df1_og
df2 = df2_og
df3 = df3_og

In [148]:
df1['age_category'] = df1['age']
df2['age_category'] = df2['age']
df3['age_category'] = df3['age']

In [149]:
for df in [df1, df2, df3]:
  df.loc[(df['age'] >= 18) & (df['age'] <= 29), 'age_category'] = 1
  df.loc[(df['age'] >= 30) & (df['age'] <= 39), 'age_category'] = 2
  df.loc[(df['age'] >= 40) & (df['age'] <= 49), 'age_category'] = 3
  df.loc[(df['age'] >= 50) & (df['age'] <= 64), 'age_category'] = 4
  df.loc[(df['age'] >= 60) & (df['age'] <= 89), 'age_category'] = 5

In [150]:
for df in [df1, df2, df3]:
  df['age'].replace(0, np.nan)
  df['age_category'].replace(0, np.nan)

In [151]:
df = pd.concat([df1, df2, df3], ignore_index = True )

In [152]:
age_category = df['age_category']
print(age_category.value_counts())

5.0    18388
2.0    14994
1.0    14360
3.0    12799
4.0    11080
Name: age_category, dtype: int64


In [153]:
polviews = df['polviews']
polviews = polviews.replace(['no answer', 'don\'t know','iap','refused','skipped on web', 'I don\'t have a job', 'dk, na, iap', 'not imputable_(2147483637)', 'not imputable_(2147483638)', 'uncodeable', 'not available in this release', 'not available in this year', 'see codebook'], np.nan)
polviews.value_counts()

moderate, middle of the road    23992
slightly conservative            9596
conservative                     9361
slightly liberal                 7900
liberal                          7623
extremely conservative           2165
extremely liberal                2081
Name: polviews, dtype: int64

In [154]:
df['polviews'] = polviews #put cleaned variable back in df

In [155]:
print(df['relig'].value_counts())

protestant                       40125
catholic                         17242
none                              9895
jewish                            1421
other                             1224
christian                         1000
buddhism                           272
muslim/islam                       201
orthodox-christian                 176
inter-nondenominational            159
hinduism                           158
other eastern religions             44
native american                     36
not imputable_(2147483638)           0
not available in this year           0
not available in this release        0
uncodeable                           0
skipped on web                       0
refused                              0
don't know                           0
not imputable_(2147483637)           0
no answer                            0
dk, na, iap                          0
I don't have a job                   0
iap                                  0
see codebook             

In [156]:
relig = df['relig']
relig = relig.replace(['not imputable_(2147483638)' ,
                       'not available in this year' ,
                       'not available in this release' ,
                       'uncodeable' ,
                       'skipped on web' ,
                       'refused' ,
                       "don't know" ,
                       'not imputable_(2147483637)' ,
                       'no answer' ,
                       'dk, na, iap' ,
                       "I don't have a job" ,
                       'iap' ,
                       'see codebook'], np.nan)
relig.value_counts()

protestant                 40125
catholic                   17242
none                        9895
jewish                      1421
other                       1224
christian                   1000
buddhism                     272
muslim/islam                 201
orthodox-christian           176
inter-nondenominational      159
hinduism                     158
other eastern religions       44
native american               36
Name: relig, dtype: int64

In [157]:
relig = relig.replace(['christian', 'inter-nondenominational', 'orthodox-christian'], 'christianity')
relig = relig.replace(['none', 'other', 'other eastern religions'], 'none/other')
relig.value_counts()

protestant         40125
catholic           17242
none/other         11163
jewish              1421
christianity        1335
buddhism             272
muslim/islam         201
hinduism             158
native american       36
Name: relig, dtype: int64

In [158]:
df['relig'] = relig #put cleaned variable back in df

In [159]:
print(df['cappun'].value_counts())

favor                            42181
oppose                           18666
don't know                           0
iap                                  0
I don't have a job                   0
dk, na, iap                          0
no answer                            0
not imputable_(2147483637)           0
not imputable_(2147483638)           0
refused                              0
skipped on web                       0
uncodeable                           0
not available in this release        0
not available in this year           0
see codebook                         0
Name: cappun, dtype: int64


In [160]:
import numpy as np
cappun = df['cappun']
cappun = cappun.replace(['no answer', 'don\'t know','iap','refused','skipped on web', 'I don\'t have a job', 'dk, na, iap', 'not imputable_(2147483637)', 'not imputable_(2147483638)', 'uncodeable', 'not available in this release', 'not available in this year', 'see codebook'], np.nan)
cappun.value_counts()

favor     42181
oppose    18666
Name: cappun, dtype: int64

In [161]:
df['cappun'] = cappun

In [162]:
print(pd.crosstab(cappun,relig),'\n')
print(pd.crosstab(cappun,polviews),'\n')
print(pd.crosstab(cappun,age_category, normalize = 'all'),'\n')

relig   protestant  catholic  jewish  none/other  buddhism  hinduism  \
cappun                                                                 
favor        24119     10144     772        5916       121        73   
oppose        9433      4378     384        3708       107        63   

relig   muslim/islam  christianity  native american  
cappun                                               
favor             93           714               16  
oppose            81           382               15   

polviews  extremely liberal  liberal  slightly liberal  \
cappun                                                   
favor                   820     3661              4551   
oppose                 1136     3361              2760   

polviews  moderate, middle of the road  slightly conservative  conservative  \
cappun                                                                        
favor                            15752                   6789          6968   
oppose                 

In [163]:
df.loc[:,['age_category','polviews','cappun']].groupby(['age_category', 'polviews']).describe()

cappun                     
                                           count unique     top  freq
age_category polviews                                                
1.0          extremely liberal               449      2  oppose   273
             liberal                        1710      2   favor   959
             slightly liberal               1813      2   favor  1101
             moderate, middle of the road   4453      2   favor  3143
             slightly conservative          1598      2   favor  1180
             conservative                   1214      2   favor   938
             extremely conservative          245      2   favor   179
2.0          extremely liberal               448      2  oppose   244
             liberal                        1558      2   favor   804
             slightly liberal               1778      2   favor  1133
             moderate, middle of the road   4535      2   favor  3321
             slightly conservative          1945      2   favor  1525
             conservative                   1551      2   favor  1216
             extremely conservative          329      2   favor   262
3.0          extremely liberal               322      2  oppose   169
             liberal                        1206      2   favor   650
             slightly liberal               1280      2   favor   809
             moderate, middle of the road   3798      2   favor  2822
             slightly conservative          1717      2   favor  1319
             conservative                   1559      2   favor  1246
             extremely conservative          329      2   favor   265
4.0          extremely liberal               289      2  oppose   178
             liberal                         944      2   favor   490
             slightly liberal                948      2   favor   609
             moderate, middle of the road   3319      2   favor  2396
             slightly conservative          1420      2   favor  1075
             conservative                   1518      2   favor  1236
             extremely conservative          372      2   favor   296
5.0          extremely liberal               420      2  oppose   251
             liberal                        1529      2  oppose   795
             slightly liberal               1441      2   favor   875
             moderate, middle of the road   5526      2   favor  3916
             slightly conservative          2166      2   favor  1639
             conservative                   2755      2   favor  2251
             extremely conservative          710      2   favor   566

In [164]:
df.loc[:,['age_category','relig','cappun']].groupby(['age_category', 'relig']).describe()

cappun                     
                              count unique     top  freq
age_category relig                                      
1.0          protestant        5809      2   favor  4028
             catholic          3066      2   favor  2105
             jewish             165      2   favor   100
             none/other        2740      2   favor  1661
             buddhism            39      2   favor    25
             hinduism            32      2   favor    18
             muslim/islam        37      2  oppose    19
             christianity       221      2   favor   131
             native american      8      2   favor     6
2.0          protestant        6407      2   favor  4663
             catholic          3122      2   favor  2259
             jewish             199      2   favor   123
             none/other        2456      2   favor  1497
             buddhism            51      2   favor    30
             hinduism            42      2   favor    23
             muslim/islam        58      2   favor    31
             christianity       304      2   favor   194
             native american      5      2  oppose     4
3.0          protestant        5824      2   favor  4275
             catholic          2524      2   favor  1784
             jewish             182      2   favor   122
             none/other        1706      2   favor  1118
             buddhism            43      2  oppose    25
             hinduism            25      2  oppose    16
             muslim/islam        40      2  oppose    21
             christianity       227      2   favor   158
             native american      9      2  oppose     7
4.0          protestant        5451      2   favor  3923
             catholic          2140      2   favor  1512
             jewish             190      2   favor   131
             none/other        1162      2   favor   740
             buddhism            35      2   favor    22
             hinduism             9      2   favor     5
             muslim/islam        25      2   favor    16
             christianity       174      2   favor   117
             native american      3      1   favor     3
5.0          protestant        9797      2   favor  7059
             catholic          3545      2   favor  2410
             jewish             404      2   favor   285
             none/other        1416      2   favor   823
             buddhism            56      2  oppose    32
             hinduism            20      2   favor    12
             muslim/islam         9      2   favor     5
             christianity       146      2   favor   103
             native american      6      2   favor     4

In [165]:
df.loc[:,['cappun','age_category']].groupby('cappun').describe()

age_category                                             
              count      mean       std  min  25%  50%  75%  max
cappun                                                          
favor       41773.0  3.070380  1.473286  1.0  2.0  3.0  5.0  5.0
oppose      18399.0  3.010761  1.501817  1.0  2.0  3.0  5.0  5.0

In [166]:
polviews.value_counts()

moderate, middle of the road    23992
slightly conservative            9596
conservative                     9361
slightly liberal                 7900
liberal                          7623
extremely conservative           2165
extremely liberal                2081
Name: polviews, dtype: int64

**polview values according to codebook**

In [175]:
categories = np.array([1,2,3,4,5,6,7])

In [176]:
df['polview_value'] = df['polviews']

In [177]:
df['polview_value'] = pd.Categorical(
   df['polview_value'], categories=categories, ordered=True)

In [178]:
df.loc[(df['polviews'] == 'extremely liberal'), 'polview_value'] = 1
df.loc[(df['polviews'] == 'liberal'), 'polview_value'] = 2
df.loc[(df['polviews'] == 'slightly liberal'), 'polview_value'] = 3
df.loc[(df['polviews'] == 'moderate, middle of the road'), 'polview_value'] = 4
df.loc[(df['polviews'] == 'slightly conservative'), 'polview_value'] = 5
df.loc[(df['polviews'] == 'conservative'), 'polview_value'] = 6
df.loc[(df['polviews'] == 'extremely conservative'), 'polview_value'] = 7

In [179]:
df['polview_value'].value_counts()

4    23992
5     9596
6     9361
3     7900
2     7623
7     2165
1     2081
Name: polview_value, dtype: int64

In [181]:
df['cappun_value'] = df['cappun']

In [182]:
df['cappun_value'] = pd.Categorical(
   df['cappun_value'], categories=categories, ordered=True)

In [183]:
df.loc[(df['cappun'] == 'favor'), 'cappun_value'] = 1
df.loc[(df['cappun'] == 'oppose'), 'cappun_value'] = 2

In [184]:
df['cappun_value'].value_counts()

1    42181
2    18666
3        0
4        0
5        0
6        0
7        0
Name: cappun_value, dtype: int64

In [185]:
df['cappun'].value_counts()

favor     42181
oppose    18666
Name: cappun, dtype: int64